# Porto Seguro Safe Driver Challenge

Kaggle Competition - Porto Seguro Safe Driver Challenge - Using H2O.ai AutoML Features

In [1]:
import os
import numpy as np
import pandas as pd
print(os.getcwd())
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

C:\Users\khark


In [3]:
os.chdir('D:/Python/Kaggle_Porto_Seguro/train')
print(os.getcwd())

D:\Python\Kaggle_Porto_Seguro\train


# Exploratory Data Analysis

In [4]:
train_data = pd.read_csv('train.csv')

In [5]:
train_data.describe()
train_data.head(10)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
5,19,0,5,1,4,0,0,0,0,0,...,4,2,0,9,0,1,0,1,1,1
6,20,0,2,1,3,1,0,0,1,0,...,3,0,0,10,0,1,0,0,1,0
7,22,0,5,1,4,0,0,1,0,0,...,7,1,3,6,1,0,1,0,1,0
8,26,0,5,1,3,1,0,0,0,1,...,4,2,1,5,0,1,0,0,0,1
9,28,1,1,1,2,0,0,0,1,0,...,3,5,0,6,0,1,0,0,1,0


In [35]:
def exploratory_analysis(data):
    print("The dimensions of the data are: " + str(data.shape))
    print("The columns / data types in this data are:")
    print(data.dtypes)
    return

def One_Hot_Encode(cols_to_transform, df):
    df_with_dummies = pd.get_dummies(df, columns = cols_to_transform )
    return df_with_dummies

In [7]:
exploratory_analysis(train_data)

The dimensions of the data are: (595212, 59)
The columns / data types in this data are:
id                  int64
target              int64
ps_ind_01           int64
ps_ind_02_cat       int64
ps_ind_03           int64
ps_ind_04_cat       int64
ps_ind_05_cat       int64
ps_ind_06_bin       int64
ps_ind_07_bin       int64
ps_ind_08_bin       int64
ps_ind_09_bin       int64
ps_ind_10_bin       int64
ps_ind_11_bin       int64
ps_ind_12_bin       int64
ps_ind_13_bin       int64
ps_ind_14           int64
ps_ind_15           int64
ps_ind_16_bin       int64
ps_ind_17_bin       int64
ps_ind_18_bin       int64
ps_reg_01         float64
ps_reg_02         float64
ps_reg_03         float64
ps_car_01_cat       int64
ps_car_02_cat       int64
ps_car_03_cat       int64
ps_car_04_cat       int64
ps_car_05_cat       int64
ps_car_06_cat       int64
ps_car_07_cat       int64
ps_car_08_cat       int64
ps_car_09_cat       int64
ps_car_10_cat       int64
ps_car_11_cat       int64
ps_car_11           int64
ps

In [8]:
# Removing Duplicates - None were found

train_data.drop_duplicates()
train_data.shape

(595212, 59)

In [25]:
# Analyze the Columns
def analyze_columns(train_data):
    data = []
    for f in train_data.columns:
        # Defining the role
        if f == 'target':
            role = 'target'
        elif f == 'id':
            role = 'id'
        else:
            role = 'input'

        if 'bin' in f or f == 'target':
            level = 'binary'
        elif 'cat' in f or f == 'id':
            level = 'nominal'
        elif train_data[f].dtype == float:
            level = 'interval'
        elif train_data[f].dtype == int:
            level = 'ordinal'

        keep = True
        if f == 'id':
            keep = False

        dtype = train_data[f].dtype

        var_dict = {
            'variable_name': f,
            'role': role,
            'level': level,
            'keep': keep,
            'dtype': dtype
        }
        data.append(var_dict)
    column_info = pd.DataFrame(data, columns=['variable_name', 'role', 'level', 'keep', 'dtype'])
    return column_info

column_info = analyze_columns(train_data)
print(column_info)

     variable_name    role     level   keep    dtype
0               id      id   nominal  False    int64
1           target  target    binary   True    int64
2        ps_ind_01   input    binary   True    int64
3    ps_ind_02_cat   input   nominal   True  float64
4        ps_ind_03   input   nominal   True    int64
5    ps_ind_04_cat   input   nominal   True  float64
6    ps_ind_05_cat   input   nominal   True  float64
7    ps_ind_06_bin   input    binary   True    int64
8    ps_ind_07_bin   input    binary   True    int64
9    ps_ind_08_bin   input    binary   True    int64
10   ps_ind_09_bin   input    binary   True    int64
11   ps_ind_10_bin   input    binary   True    int64
12   ps_ind_11_bin   input    binary   True    int64
13   ps_ind_12_bin   input    binary   True    int64
14   ps_ind_13_bin   input    binary   True    int64
15       ps_ind_14   input    binary   True    int64
16       ps_ind_15   input    binary   True    int64
17   ps_ind_16_bin   input    binary   True   

In [31]:
# Understanding the numeric variables

interval_vars = list(column_info[column_info["level"] == "interval"]["variable_name"])
train_data_interval = train_data.loc[:,interval_vars]
train_data_interval.describe()

,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,...,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.610991,0.439184,0.894047,2.346100,0.379947,0.813265,0.374691,3.065899,0.449756,0.449589,...,1.885886,7.689445,3.005823,9.225904,2.339034,8.433590,5.441382,1.441918,2.872288,7.539026
std,0.287643,0.404264,0.312581,0.832491,0.058300,0.224588,0.043947,0.731366,0.287198,0.286893,...,1.134927,1.334312,1.414564,1.459672,1.246949,2.904597,2.332871,1.202963,1.694887,2.746652
min,0.000000,0.000000,0.061237,0.000000,0.100000,0.250619,0.109545,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.200000,0.666615,2.000000,0.316228,0.670867,0.353553,2.828427,0.200000,0.200000,...,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000
50%,0.700000,0.300000,0.894047,3.000000,0.374166,0.765811,0.374691,3.316625,0.500000,0.400000,...,2.000000,8.000000,3.000000,9.000000,2.000000,8.000000,5.000000,1.000000,3.000000,7.000000
75%,0.900000,0.600000,1.000000,3.000000,0.400000,0.906190,0.396485,3.605551,0.700000,0.700000,...,3.000000,9.000000,4.000000,10.000000,3.000000,10.000000,7.000000,2.000000,4.000000,9.000000
max,0.900000,1.800000,4.037945,3.000000,1.264911,3.720626,0.636396,3.741657,0.900000,0.900000,...,6.000000,10.000000,9.000000,12.000000,7.000000,25.000000,19.000000,10.000000,13.000000,23.000000


# Missing Values

In [11]:
def missing_vars():
    vars_with_missing = []
    for f in train_data.columns:
        missings = train_data[train_data[f] == -1][f].count()
        if missings > 0:
            vars_with_missing.append(f)
            missings_perc = missings/train_data.shape[0]
            print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
    print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))
    return vars_with_missing
missing_vars = missing_vars()

Variable ps_ind_02_cat has 216 records (0.04%) with missing values
Variable ps_ind_04_cat has 83 records (0.01%) with missing values
Variable ps_ind_05_cat has 5809 records (0.98%) with missing values
Variable ps_reg_03 has 107772 records (18.11%) with missing values
Variable ps_car_01_cat has 107 records (0.02%) with missing values
Variable ps_car_02_cat has 5 records (0.00%) with missing values
Variable ps_car_03_cat has 411231 records (69.09%) with missing values
Variable ps_car_05_cat has 266551 records (44.78%) with missing values
Variable ps_car_07_cat has 11489 records (1.93%) with missing values
Variable ps_car_09_cat has 569 records (0.10%) with missing values
Variable ps_car_11 has 5 records (0.00%) with missing values
Variable ps_car_12 has 1 records (0.00%) with missing values
Variable ps_car_14 has 42620 records (7.16%) with missing values
In total, there are 13 variables with missing values


In [13]:
print(missing_vars)

['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_05_cat', 'ps_car_07_cat', 'ps_car_09_cat', 'ps_car_11', 'ps_car_12', 'ps_car_14']


In [14]:
# Imputing All other Values using mean imputation
mean_imputer = Imputer(missing_values=-1, strategy='mean', axis=0)
for var in missing_vars:
    train_data[var] = mean_imputer.fit_transform(train_data[[var]]).ravel()

In [15]:
# Remove the variables ps_car_03_cat and ps_car_05_cat
train_data.drop(['ps_car_03_cat', 'ps_car_05_cat'], inplace=True, axis=1)

In [33]:
# One Hot Encoding the Rest of the Binary / Categorical Variables

column_df = column_info.loc[column_info["level"] != "interval"]
print(column_df)

     variable_name    role    level   keep    dtype
0               id      id  nominal  False    int64
1           target  target   binary   True    int64
2        ps_ind_01   input   binary   True    int64
3    ps_ind_02_cat   input  nominal   True  float64
4        ps_ind_03   input  nominal   True    int64
5    ps_ind_04_cat   input  nominal   True  float64
6    ps_ind_05_cat   input  nominal   True  float64
7    ps_ind_06_bin   input   binary   True    int64
8    ps_ind_07_bin   input   binary   True    int64
9    ps_ind_08_bin   input   binary   True    int64
10   ps_ind_09_bin   input   binary   True    int64
11   ps_ind_10_bin   input   binary   True    int64
12   ps_ind_11_bin   input   binary   True    int64
13   ps_ind_12_bin   input   binary   True    int64
14   ps_ind_13_bin   input   binary   True    int64
15       ps_ind_14   input   binary   True    int64
16       ps_ind_15   input   binary   True    int64
17   ps_ind_16_bin   input   binary   True    int64
18   ps_ind_

In [37]:
# One-Hot Encode the remaining variables for Model Building

cols_to_transform = list(column_df["variable_name"])
cols_to_transform.remove("id")
cols_to_transform.remove("target")

train_data_2 = One_Hot_Encode(cols_to_transform, train_data)

# Model Building with H2O

In [38]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 5 hours 29 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.5
H2O cluster version age:,20 days
H2O cluster name:,H2O_from_python_khark_abrqd9
H2O cluster total nodes:,1
H2O cluster free memory:,5.223 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [41]:
train_data_3 = h2o.H2OFrame(train_data_2)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [42]:
train_data_3.describe()

Rows:595212
Cols:275




,id,target,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_ind_01_0,ps_ind_01_1,ps_ind_01_2,ps_ind_01_3,ps_ind_01_4,ps_ind_01_5,ps_ind_01_6,ps_ind_01_7,ps_ind_02_cat_1.0,ps_ind_02_cat_1.3597990574726553,ps_ind_02_cat_2.0,ps_ind_02_cat_3.0,ps_ind_02_cat_4.0,ps_ind_03_0,ps_ind_03_1,ps_ind_03_2,ps_ind_03_3,ps_ind_03_4,ps_ind_03_5,ps_ind_03_6,ps_ind_03_7,ps_ind_03_8,ps_ind_03_9,ps_ind_03_10,ps_ind_03_11,ps_ind_04_cat_0.0,ps_ind_04_cat_0.4169919462839149,ps_ind_04_cat_1.0,ps_ind_05_cat_0.0,ps_ind_05_cat_0.41903756852272556,ps_ind_05_cat_1.0,ps_ind_05_cat_2.0,ps_ind_05_cat_3.0,ps_ind_05_cat_4.0,ps_ind_05_cat_5.0,ps_ind_05_cat_6.0,ps_ind_06_bin_0,ps_ind_06_bin_1,ps_ind_07_bin_0,ps_ind_07_bin_1,ps_ind_08_bin_0,ps_ind_08_bin_1,ps_ind_09_bin_0,ps_ind_09_bin_1,ps_ind_10_bin_0,ps_ind_10_bin_1,ps_ind_11_bin_0,ps_ind_11_bin_1,ps_ind_12_bin_0,ps_ind_12_bin_1,ps_ind_13_bin_0,ps_ind_13_bin_1,ps_ind_14_0,ps_ind_14_1,ps_ind_14_2,ps_ind_14_3,ps_ind_14_4,ps_ind_15_0,ps_ind_15_1,ps_ind_15_2,ps_ind_15_3,ps_ind_15_4,ps_ind_15_5,ps_ind_15_6,ps_ind_15_7,ps_ind_15_8,ps_ind_15_9,ps_ind_15_10,ps_ind_15_11,ps_ind_15_12,ps_ind_15_13,ps_ind_16_bin_0,ps_ind_16_bin_1,ps_ind_17_bin_0,ps_ind_17_bin_1,ps_ind_18_bin_0,ps_ind_18_bin_1,ps_car_01_cat_0.0,ps_car_01_cat_1.0,ps_car_01_cat_2.0,ps_car_01_cat_3.0,ps_car_01_cat_4.0,ps_car_01_cat_5.0,ps_car_01_cat_6.0,ps_car_01_cat_7.0,ps_car_01_cat_8.0,ps_car_01_cat_8.297604624394015,ps_car_01_cat_9.0,ps_car_01_cat_10.0,ps_car_01_cat_11.0,ps_car_02_cat_0.0,ps_car_02_cat_0.8299465564081068,ps_car_02_cat_1.0,ps_car_04_cat_0,ps_car_04_cat_1,ps_car_04_cat_2,ps_car_04_cat_3,ps_car_04_cat_4,ps_car_04_cat_5,ps_car_04_cat_6,ps_car_04_cat_7,ps_car_04_cat_8,ps_car_04_cat_9,ps_car_06_cat_0,ps_car_06_cat_1,ps_car_06_cat_2,ps_car_06_cat_3,ps_car_06_cat_4,ps_car_06_cat_5,ps_car_06_cat_6,ps_car_06_cat_7,ps_car_06_cat_8,ps_car_06_cat_9,ps_car_06_cat_10,ps_car_06_cat_11,ps_car_06_cat_12,ps_car_06_cat_13,ps_car_06_cat_14,ps_car_06_cat_15,ps_car_06_cat_16,ps_car_06_cat_17,ps_car_07_cat_0.0,ps_car_07_cat_0.9476207036556723,ps_car_07_cat_1.0,ps_car_08_cat_0,ps_car_08_cat_1,ps_car_09_cat_0.0,ps_car_09_cat_1.0,ps_car_09_cat_1.3311179985302106,ps_car_09_cat_2.0,ps_car_09_cat_3.0,ps_car_09_cat_4.0,ps_car_10_cat_0,ps_car_10_cat_1,ps_car_10_cat_2,ps_car_11_cat_1,ps_car_11_cat_2,ps_car_11_cat_3,ps_car_11_cat_4,ps_car_11_cat_5,ps_car_11_cat_6,ps_car_11_cat_7,ps_car_11_cat_8,ps_car_11_cat_9,ps_car_11_cat_10,ps_car_11_cat_11,ps_car_11_cat_12,ps_car_11_cat_13,ps_car_11_cat_14,ps_car_11_cat_15,ps_car_11_cat_16,ps_car_11_cat_17,ps_car_11_cat_18,ps_car_11_cat_19,ps_car_11_cat_20,ps_car_11_cat_21,ps_car_11_cat_22,ps_car_11_cat_23,ps_car_11_cat_24,ps_car_11_cat_25,ps_car_11_cat_26,ps_car_11_cat_27,ps_car_11_cat_28,ps_car_11_cat_29,ps_car_11_cat_30,ps_car_11_cat_31,ps_car_11_cat_32,ps_car_11_cat_33,ps_car_11_cat_34,ps_car_11_cat_35,ps_car_11_cat_36,ps_car_11_cat_37,ps_car_11_cat_38,ps_car_11_cat_39,ps_car_11_cat_40,ps_car_11_cat_41,ps_car_11_cat_42,ps_car_11_cat_43,ps_car_11_cat_44,ps_car_11_cat_45,ps_car_11_cat_46,ps_car_11_cat_47,ps_car_11_cat_48,ps_car_11_cat_49,ps_car_11_cat_50,ps_car_11_cat_51,ps_car_11_cat_52,ps_car_11_cat_53,ps_car_11_cat_54,ps_car_11_cat_55,ps_car_11_cat_56,ps_car_11_cat_57,ps_car_11_cat_58,ps_car_11_cat_59,ps_car_11_cat_60,ps_car_11_cat_61,ps_car_11_cat_62,ps_car_11_cat_63,ps_car_11_cat_64,ps_car_11_cat_65,ps_car_11_cat_66,ps_car_11_cat_67,ps_car_11_cat_68,ps_car_11_cat_69,ps_car_11_cat_70,ps_car_11_cat_71,ps_car_11_cat_72,ps_car_11_cat_73,ps_car_11_cat_74,ps_car_11_cat_75,ps_car_11_cat_76,ps_car_11_cat_77,ps_car_11_cat_78,ps_car_11_cat_79,ps_car_11_cat_80,ps_car_11_cat_81,ps_car_11_cat_82,ps_car_11_cat_83,ps_car_11_cat_84,ps_car_11_cat_85,ps_car_11_cat_86,ps_car_11_cat_87,ps_car_11_cat_88,ps_car_11_cat_89,ps_car_11_cat_90,ps_car_11_cat_91,ps_car_11_

In [49]:
# Removing the Variables that we don't want

y = "target"
train_data_3[y] = train_data_3[y].asfactor()
x = train_data_3.columns
x.remove(y)
x.remove("id")

In [50]:
aml = H2OAutoML(max_models = 10, seed = 1, max_runtime_secs = 120)
aml.train(x = x, y = y, training_frame = train_data_3)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [51]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
DRF_0_AutoML_20180829_211149,0.588644,0.158305,0.44605,0.188555,0.035553


In [54]:
h2o.cluster().shutdown()

This is my first pass at using H2O's AutoML software for a prediction against the Porto Seguro Dataset
Improvements can include:
 - Longer Runtimes for the AutoML feature
 - Addressing challenges in class imbalance (multitude of tools available for this)